In [1]:
# TODO: Slice signal and store it as processed features. Keep instances dates as metadata.
from src.config import get_logger

import pandas as pd
from pathlib import Path
# import reservoirpy as rpy
import yaml
from src import paths
# from src.data.utils import create_output_path
from src.features.slice_time_series import create_training_data_for_multiple_pixels


In [2]:
params_path: Path = paths.config_dir("params.yaml")

  # rpy.verbosity(0)

with open(params_path, "r") as file:
    params = yaml.safe_load(file)

selected_band: str = params["selected_band"]
esn_features_dim: int = params["esn_features_dim"]
validation_limit_date: str = params["validation_limit_date"]

In [3]:
denoised_train_signal_filename = "_".join(
        ["denoised_train_signal_filtered_dataset", selected_band])
denoised_train_signal_filename += ".csv"
# denoised_signal_path = denoised_esn_signal_dir / denoised_esn_signal_filename

denoised_esn_signal_df = pd.read_csv(paths.features_dir(
    denoised_train_signal_filename), index_col=["ID", "IDpix"])
denoised_esn_signal_df.columns = pd.to_datetime(
    denoised_esn_signal_df.columns)

In [10]:
train_size = denoised_esn_signal_df.loc[:, denoised_esn_signal_df.columns < pd.Timestamp(validation_limit_date)].shape[1]

In [18]:
print(denoised_esn_signal_df.columns[train_size -1])


2009-12-27 00:00:00


In [11]:
denoised_ts = denoised_esn_signal_df.to_numpy()


In [12]:
Xtrain, ytrain, Xval, yval = create_training_data_for_multiple_pixels(
        denoised_ts, num_features=esn_features_dim, train_size=train_size)

ytrain = ytrain.reshape(-1, 1)
yval = yval.reshape(-1, 1)


In [21]:
ytrain.shape

(56028, 1)

In [20]:
Xtrain.shape

(56028, 104)